In [ ]:
!pip install pyspark
!pip install neo4j
!pip install pandas

import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b0354400cc67f63e6bc73724721912438a9702dac178a6f5d95f3c82b8a065db
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=7107d0d4f1a941066c8bc4b67001ec6697e91377de20d1e5744a45807f5969fb
  Stored in directory: /root/.cache/pip/wheels/cb/12/66/764554d079caad4b9a11a02cfc0d200dd876d12935b9cf7e64
Successfully built neo4j


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ejemplo de PySpark en Jupyter Notebook") \
    .getOrCreate()

# Obtener el SparkContext
sc = spark.sparkContext

In [ ]:
graph = [(1,11,2),(1,11,3),(2,11,3),(3,11,2),(3,11,4),(4,11,1),(4,11,2),(4,11,3),(4,12,5),(5,12,1),(5,12,2),(5,12,6)]
#graph = [(2,11,2), (1,11,2), (2,11,1)]

In [ ]:
B = 2

In [ ]:
def hash(x):
    return x % B

In [ ]:
def map_pdm(x, arista, y):

    x_hash = hash(x)
    y_hash = hash(y)

    l = []
    for i in range(B):
        l.append(((x_hash, y_hash, i), (x, arista, y)))
        l.append(((i, x_hash, y_hash), (x, arista, y)))
        l.append(((y_hash, i, x_hash), (x, arista, y)))

    return set(l)

In [ ]:
datos_rdd = sc.parallelize(graph)

### Pasarlo por map

In [ ]:
rdd_map = datos_rdd.flatMap(lambda dato: map_pdm(*dato))

rdd_map.collect()[0:2]

[((1, 0, 0), (1, 11, 2)), ((0, 1, 1), (1, 11, 2))]

### Reduce


In [ ]:
rdd_reduce = rdd_map.groupByKey().mapValues(list)
rdd_reduce.collect()[0]

((1, 0, 0),
 [(1, 11, 2),
  (2, 11, 3),
  (3, 11, 2),
  (3, 11, 4),
  (4, 11, 1),
  (4, 11, 2),
  (4, 11, 3),
  (4, 12, 5),
  (5, 12, 2),
  (5, 12, 6)])

In [ ]:
def triangulo(nodos):

  triangulos_detectados = []
  for i in range(len(nodos)):
    nodo_actual = nodos[i]

    for j in range(i + 1, len(nodos)):

      nodo_sig = nodos[j]

      for k in range(j + 1, len(nodos)):
        nodo_sub_sig = nodos[k]

        if nodo_actual[2] == nodo_sig[0] and nodo_sig[2] == nodo_sub_sig[0] and nodo_actual[0] == nodo_sub_sig[2]:
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

        elif nodo_actual[2] == nodo_sub_sig[0] and nodo_sig[0] == nodo_sub_sig[2] and nodo_actual[0] == nodo_sig[2]:
          triangulos_detectados.append((nodo_actual[0], nodo_sig[0], nodo_sub_sig[0]))

  return triangulos_detectados

In [ ]:
final = rdd_reduce.map(lambda nodos: (nodos[0], triangulo(nodos[1])))
final.collect()

[((1, 0, 0), [(2, 3, 4)]),
 ((0, 1, 0), [(2, 3, 4)]),
 ((0, 0, 1), [(2, 3, 4)]),
 ((1, 1, 1), []),
 ((0, 1, 1), [(1, 3, 4)]),
 ((1, 0, 1), [(1, 3, 4)]),
 ((1, 1, 0), [(1, 3, 4)]),
 ((0, 0, 0), [])]

In [ ]:
dicc = {}
for i in final.collect():
    dicc[i[0]] = i[1]

In [ ]:
dicc

{(1, 0, 0): [(2, 3, 4)],
 (0, 1, 0): [(2, 3, 4)],
 (0, 0, 1): [(2, 3, 4)],
 (1, 1, 1): [],
 (0, 1, 1): [(1, 3, 4)],
 (1, 0, 1): [(1, 3, 4)],
 (1, 1, 0): [(1, 3, 4)],
 (0, 0, 0): []}

In [ ]:
from neo4j import GraphDatabase

URI = "neo4j+s://06ae1fa1.databases.neo4j.io"
AUTH = ("neo4j","QjwVk3kN-OI5bTt-fg6LZy-F4LMHCpL9HFxuvSuq-OE")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    try:
        session.run("RETURN 1")
        print("Connection to Neo4j established successfully!")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

Connection to Neo4j established successfully!


### Función a cambiar

In [ ]:
def get_data_from_neo4j():
    with driver.session() as session:
        result = session.run("""
        MATCH (n1:Node)-[r:RELATED]->(n2:Node)
        RETURN n1.id AS id_form, n2.id AS id_to, r.weight AS weight
        """)
        data = [record.data() for record in result]
    return data

# Obtiene los datos de Neo4j
neo4j_data = get_data_from_neo4j()

In [ ]:
tuples = [(d['id_form'], d['weight'], d['id_to']) for d in neo4j_data]

rdd = sc.parallelize(tuples)

In [ ]:
rdd.take(5)

[(1, 1, 2), (1, 7, 3), (4, 3, 5), (6, 5, 7), (6, 14137, 8)]

In [ ]:
driver.close()